<a href="https://colab.research.google.com/github/ethvedbitdesjan/SummerResearch/blob/main/SimpleClassifiers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This get the RAPIDS-Colab install files and test check your GPU.  Run this and the next cell only.
# Please read the output of this cell.  If your Colab Instance is not RAPIDS compatible, it will warn you and give you remediation steps.
!git clone https://github.com/rapidsai/rapidsai-csp-utils.git
!python rapidsai-csp-utils/colab/pip-install.py

Cloning into 'rapidsai-csp-utils'...
remote: Enumerating objects: 390, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 390 (delta 89), reused 51 (delta 51), pack-reused 269
Receiving objects: 100% (390/390), 107.11 KiB | 5.95 MiB/s, done.
Resolving deltas: 100% (191/191), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 7.4 MB/s eta 0:00:00
***********************************************************************
Woo! Your instance has the right kind of GPU, a Tesla T4!
We will now install RAPIDS cuDF, cuML, and cuGraph via pip! 
Please stand by, should be quick...
***********************************************************************

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.

In [1]:
# Critical imports
import os
import numpy as np
import pandas as pd
from PIL import Image
import random
import cv2
import copy

In [2]:
from google.colab import drive
drive.mount('content/')

Mounted at content/


In [3]:
%cd /content/content/MyDrive/SummerResearch

/content/content/MyDrive/SummerResearch


In [40]:
funny_set = set()
not_funny_set = set()
with open('labelled_data/funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        funny_set.add(line.strip())
with open('labelled_data/not_funny_combined1.csv', 'r') as f:
    lines = f.readlines()
    for line in lines:
        not_funny_set.add(line.strip())

In [41]:
train_data = []
train_funny_data = []
train_not_funny_data = []
count = 0
added = set()
for file in os.listdir('Funny_Originals_Final'):
  img_array = cv2.imread(os.path.join('Funny_Originals_Final', file))
  img_array_resized = cv2.resize(img_array, (150, 150))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array_resized.reshape(-1).astype(np.float64), 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array_resized.reshape(-1).astype(np.float64), 0))
  count += 1
for file in os.listdir('Non-Funny_Modified_Final'):
  img_array = cv2.imread(os.path.join('Non-Funny_Modified_Final', file))
  img_array_resized = cv2.resize(img_array, (150, 150))
  if file in funny_set:
    added.add(file)
    train_funny_data.append((img_array_resized.reshape(-1).astype(np.float64), 1))
  if file in not_funny_set:
    added.add(file)
    train_not_funny_data.append((img_array_resized.reshape(-1).astype(np.float64), 0))

In [42]:
random.seed()

In [43]:
print(len(train_funny_data), len(train_not_funny_data))
train_not_funny_data1 = random.sample(train_not_funny_data, k = len(train_funny_data)+10)
train_data = copy.deepcopy(train_funny_data) + copy.deepcopy(train_not_funny_data1)
len(train_data), len(train_funny_data), len(train_not_funny_data), len(train_not_funny_data1)

1133 1560


(2276, 1133, 1560, 1143)

In [44]:
random.shuffle(train_data)

In [45]:
x = []
y = []
for xi, yi in train_data:
  x.append(xi)
  y.append(yi)
x = np.array(x)
y = np.array(y)

In [46]:
from sklearn.svm import SVC
svm = SVC()

In [47]:
x.shape, y.shape

((2276, 67500), (2276,))

In [48]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2)

In [49]:
svm.fit(x_train, y_train)

SVC()

In [50]:
from sklearn.metrics import accuracy_score
y_pred = svm.predict(x_test)
accuracy_score(y_pred, y_test)

0.5087719298245614

Accuracy^

In [51]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_pred, y_test), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

,Test Not Funny,Test Funny
Pred Not Funny,115,118
Pred Funny,106,117


In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(
        x_train,
        y_train    )

dtest = xgb.DMatrix(
        x_test,
        y_test    )

## Train the model
trained_model = xgb.train(
                        {
                          'learning_rate': 0.1,
                          'colsample_bytree' : 0.3,
                          'max_depth': 5,
                          'objective': 'reg:linear',
                          'n_estimators':10,
                          'alpha' : 10,
                          'silent': True,
                          'verbose_eval': True,
                          'tree_method':'gpu_hist',
                        },
                        dtrain,
                        num_boost_round=100, evals=[(dtrain, 'train')])

## Predict the model
prediction = trained_model.predict(dtest)

[21:55:25] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[21:55:26] WARNING: ../src/learner.cc:767: 
Parameters: { "n_estimators", "silent", "verbose_eval" } are not used.

[0]	train-rmse:0.49140
[1]	train-rmse:0.48408
[2]	train-rmse:0.47704
[3]	train-rmse:0.46953
[4]	train-rmse:0.46232
[5]	train-rmse:0.45585
[6]	train-rmse:0.44928
[7]	train-rmse:0.44317
[8]	train-rmse:0.43697
[9]	train-rmse:0.43167
[10]	train-rmse:0.42607
[11]	train-rmse:0.42052
[12]	train-rmse:0.41657
[13]	train-rmse:0.41142
[14]	train-rmse:0.40544
[15]	train-rmse:0.40067
[16]	train-rmse:0.39589
[17]	train-rmse:0.39077
[18]	train-rmse:0.38616
[19]	train-rmse:0.38193
[20]	train-rmse:0.37821
[21]	train-rmse:0.37472
[22]	train-rmse:0.37092
[23]	train-rmse:0.36685
[24]	train-rmse:0.36308
[25]	train-rmse:0.35916
[26]	train-rmse:0.35569
[27]	train-rmse:0.35224
[28]	train-rmse:0.34849
[29]	train-rmse:0.34484
[30]	train-rmse:0.34171
[31]	train-rmse:0.33877
[32]	tr

In [ ]:
y_test = y_test.astype(np.float64)

## Test prediction wih RMSE, compare it to sklearn and pandas.
from sklearn.metrics import mean_squared_error, accuracy_score
rmse = np.sqrt(mean_squared_error(y_test, prediction))

y_pred_nn = (prediction > 0.5).astype(np.float32)

## RMSE for the price prediction is per 1000$.  Let's see what we get...
print("RMSE: ", rmse, "Accuracy: ", accuracy_score(y_pred_nn, y_test))

RMSE:  0.5166712788850096 Accuracy:  0.5295315682281059


In [ ]:
y_pred_nn.shape

(491,)

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_pred_nn, y_test), columns=['Test Not Funny', 'Test Funny'], index=['Pred Not Funny', 'Pred Funny'])

,Test Not Funny,Test Funny
Pred Not Funny,132,108
Pred Funny,123,128
